In [1]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split

# 데이터 불러오기

In [2]:
df = pd.read_csv('분석용데이터_utf_spi수정완료.csv')

In [3]:
#Date를 index로 넣어줌
df.index = df['Date']
df.drop(['Date'], axis = 1, inplace = True)

In [4]:
df

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,spi
Date,,,,,,,,
201101월,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125,81.3
201102월,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906,82.5
201103월,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289,83.4
201104월,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791,83.9
201105월,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709,84.1
...,...,...,...,...,...,...,...,...
202008월,3100.4,2.19,0.50,0.7,2326.17,1187.94,1221.939746,113.1
202009월,3115.2,2.24,0.50,1.0,2327.89,1164.65,1208.597571,114.0
202010월,3152.8,2.24,0.50,0.1,2267.15,1136.49,1001.099665,115.6


# 전처리

In [5]:
df['next_spi'] = df['spi'].shift(-1)
df['next_spi'][-1] = 124.9

In [6]:
#전처리에 사용되었던 더미 변수들 drop
df.drop(['spi'], axis = 1, inplace = True)

In [7]:
X = df.copy()
y = X.pop('next_spi')

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [10]:
X_train_new = X_train.drop(['효준감성점수'], axis = 1)
X_test_new = X_test.drop(['효준감성점수'], axis = 1)

In [11]:
X_train

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수
Date,,,,,,,
201504월,2148.1,1.99,1.75,0.4,2127.17,1076.34,2212.598416
202006월,3077.3,2.22,0.50,0.0,2108.33,1199.28,1027.105971
201912월,2912.0,1.95,1.25,0.7,2197.67,1154.07,873.706855
201602월,2285.3,1.95,1.50,1.1,1916.66,1237.24,1422.961939
201104월,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791
...,...,...,...,...,...,...,...
201608월,2377.3,1.65,1.25,0.5,2034.65,1115.29,1454.840123
201605월,2312.8,1.91,1.50,0.8,1983.40,1188.39,1619.125596
202010월,3152.8,2.24,0.50,0.1,2267.15,1136.49,1001.099665


# 효준이 제외 기본 성능

In [12]:
from sklearn.ensemble import RandomForestRegressor

In [13]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

In [14]:
regressor.fit(X_train_new, y_train)  

RandomForestRegressor(random_state=0)

In [15]:
y_pred = regressor.predict(X_test_new)  # test the output by changing values

In [16]:
from sklearn.metrics import mean_squared_error


In [17]:
RMSE = mean_squared_error(y_test, y_pred)**0.5

In [18]:
RMSE

1.422598783916282

# 효준이 포함 성능

In [19]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

In [20]:
regressor.fit(X_train, y_train)  

RandomForestRegressor(random_state=0)

In [21]:
y_pred_new = regressor.predict(X_test)  # test the output by changing values

In [22]:
RMSE_new = mean_squared_error(y_test, y_pred_new)**0.5

In [23]:
RMSE_new

1.517531043065262